# Astrometry

In [2]:
# make a table in LaTeX format for easy pasting

import h5py
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Plots.model_plot import ModelPlot

# define filters and system names
names = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

# table format:
# Lens System Name, RA_lens, Dec_lens, RA_im1, Dec_im1, Ra_im2, Dec_im2, Image Separation [arcsec]
for i, name in enumerate(names):
    filename = f"../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"

    with h5py.File(filename, "r") as f:
        kwargs = {}
        for key in f:
            kwargs[key] = f[key][()]

    if name=='J0602-4335':
        ra_lens = kwargs['center_x_0']
        dec_lens = kwargs['center_y_0']        
    else:
        ra_lens = kwargs['center_x']
        dec_lens = kwargs['center_y']

    ra_im1 = kwargs['image_position'][0][0] - ra_lens
    dec_im1 = kwargs['image_position'][1][0] - dec_lens

    ra_im2 = kwargs['image_position'][0][1] - ra_lens
    dec_im2 = kwargs['image_position'][1][1] - dec_lens

    image_sep = np.sqrt((kwargs['image_position'][0][0] - kwargs['image_position'][0][1])**2 + (kwargs['image_position'][1][0] - kwargs['image_position'][1][1])**2)

    # print the data in LaTeX format
    print(f"{name} & {ra_lens:.3f} & {dec_lens:.3f} & {ra_im1:.3f} & {dec_im1:.3f} & {ra_im2:.3f} & {dec_im2:.3f} & {image_sep:.3f} \\\\")

J0407-5006 & 3.455 & -0.678 & -0.157 & -0.257 & 0.734 & 1.214 & 1.721 \\
J0602-4335 & 3.946 & 0.013 & -0.764 & -0.926 & 0.416 & 0.470 & 1.828 \\
J0806+2006 & 4.191 & -0.157 & -0.929 & 0.501 & 0.390 & -0.185 & 1.486 \\
J1442+4055 & 4.234 & -0.265 & -1.300 & 0.394 & 0.758 & -0.162 & 2.132 \\
J1515+1511 & 4.556 & -0.214 & -1.403 & 0.851 & 0.265 & -0.266 & 2.008 \\
J1620+1203 & 3.274 & -0.254 & -0.339 & -0.398 & 1.768 & 1.462 & 2.810 \\
J2325-5229 & 3.795 & -0.364 & -0.488 & 1.657 & 0.911 & -0.792 & 2.821 \\


# Magnitude Table

In [3]:
import h5py
import numpy as np
import os

# Define system names
names = ['J0407-5006', 'J0602-4335', 'J0806+2006', 
         'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def flux_ratio_and_sigma(fA, fB, sigmaA, sigmaB):
    """Compute flux ratio fA/fB with asymmetric errors."""
    ratio = fA / fB

    # upper and lower separately
    sigmaA_upper, sigmaA_lower = sigmaA
    sigmaB_upper, sigmaB_lower = sigmaB

    # upper error propagation
    sigmaR_upper = ratio * np.sqrt((sigmaA_upper / fA)**2 + (sigmaB_lower / fB)**2)

    # lower error propagation
    sigmaR_lower = ratio * np.sqrt((sigmaA_lower / fA)**2 + (sigmaB_upper / fB)**2)

    return ratio, sigmaR_upper, sigmaR_lower

def format_quantity(val, upper, lower, decimals=2):
    return f"${val:.{decimals}f}^{{+{upper:.{decimals}f}}}_{{-{lower:.{decimals}f}}}$"

def mag_sigma(flux, sigma_flux, zp_err_mag=0.018):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]

    # Zeropoint mag error converted to flux error
    zp_flux_err = 0.921 * flux * zp_err_mag  

    # Total flux uncertainties
    total_upper = np.sqrt(sigma_upper**2 + zp_flux_err**2)
    total_lower = np.sqrt(sigma_lower**2 + zp_flux_err**2)

    # Convert to magnitude uncertainties
    mag_err_upper = (2.5 / np.log(10)) * (total_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (total_lower / flux)

    return mag_err_upper, mag_err_lower

def format_mag(mag, sigmas):
    return f"${mag:.2f}^{{+{sigmas[0]:.2f}}}_{{-{sigmas[1]:.2f}}}$"

# --- LaTeX header ---
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lccccc}")
print("\\hline")
print("Lens System & Filter & Lens & Image A & Image B & $f_A/f_B$ \\\\")
print("\\hline")

# --- Loop over systems ---
for name in names:
    row_started = False  # only print lens name on first row
    for filt in ["F814W", "F475X", "F160W"]:
        filename = f"../cutout_data/{name}/{filt}/{name}_{filt}_final.hdf5"

        if os.path.exists(filename):
            with h5py.File(filename, "r") as f:
                kwargs = {key: f[key][()] for key in f}

            # mags
            lens_mag = format_mag(kwargs['lens_mag'], 
                                  mag_sigma(kwargs['lens_flux'], kwargs['lens_flux_sigma']))
            im1_mag = format_mag(kwargs['image_mags'][0], 
                                 mag_sigma(kwargs['image_fluxes'][0], kwargs['image_fluxes_sigma'][0]))
            im2_mag = format_mag(kwargs['image_mags'][1], 
                                 mag_sigma(kwargs['image_fluxes'][1], kwargs['image_fluxes_sigma'][1]))

            # flux ratio
            fA, fB = kwargs['image_fluxes'][0], kwargs['image_fluxes'][1]
            sigmaA = kwargs['image_fluxes_sigma'][0]
            sigmaB = kwargs['image_fluxes_sigma'][1]

            ratio, r_up, r_low = flux_ratio_and_sigma(fA, fB, sigmaA, sigmaB)
            flux_ratio = format_quantity(ratio, r_up, r_low, decimals=2)
        else:
            lens_mag = im1_mag = im2_mag = flux_ratio = "--"

        # print row
        if not row_started:
            print(f"{name} & {filt} & {lens_mag} & {im1_mag} & {im2_mag} & {flux_ratio} \\\\")
            row_started = True
        else:
            print(f" & {filt} & {lens_mag} & {im1_mag} & {im2_mag} & {flux_ratio} \\\\")

    print("\\hline")

print("\\end{tabular*}")


\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lccccc}
\hline
Lens System & Filter & Lens & Image A & Image B & $f_A/f_B$ \\
\hline
J0407-5006 & F814W & $20.35^{+0.02}_{-0.02}$ & $19.39^{+0.02}_{-0.02}$ & $18.01^{+0.02}_{-0.02}$ & $0.28^{+0.00}_{-0.00}$ \\
 & F475X & $22.39^{+0.02}_{-0.02}$ & $19.81^{+0.02}_{-0.02}$ & $18.66^{+0.02}_{-0.02}$ & $0.35^{+0.00}_{-0.00}$ \\
 & F160W & $18.56^{+0.02}_{-0.02}$ & $19.59^{+0.03}_{-0.03}$ & $17.97^{+0.02}_{-0.02}$ & $0.24^{+0.01}_{-0.01}$ \\
\hline
J0602-4335 & F814W & $19.01^{+0.02}_{-0.02}$ & $19.66^{+0.02}_{-0.02}$ & $19.46^{+0.02}_{-0.02}$ & $0.84^{+0.00}_{-0.00}$ \\
 & F475X & $20.80^{+0.02}_{-0.02}$ & $19.87^{+0.02}_{-0.02}$ & $19.69^{+0.02}_{-0.02}$ & $0.85^{+0.00}_{-0.00}$ \\
 & F160W & $17.80^{+0.02}_{-0.02}$ & $19.20^{+0.02}_{-0.02}$ & $18.98^{+0.02}_{-0.02}$ & $0.82^{+0.01}_{-0.01}$ \\
\hline
J0806+2006 & F814W & $19.88^{+0.02}_{-0.02}$ & $19.25^{+0.02}_{-0.02}$ & $20.07^{+0.02}_{-0.02}$ & $2.13^{+0.00}_{-0.00}$ \\
 & F475X & $23.

# Lens Light Parameter Table

In [4]:
import pickle
import numpy as np
import os

names = ['J0407-5006', 'J0602-4335', 'J0806+2006',
         'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

def format_val(med, lo, up, fmt=".2f"):
    return f"${med:{fmt}}^{{+{up:{fmt}}}}_{{-{lo:{fmt}}}}$"

def extract_profile(samples, param_names, param_mcmc):
    """Return R, n, q, phi chains for one profile."""
    idx = [param_mcmc.index(p) for p in param_names]
    r_chain = samples[:, idx[0]]
    n_chain = samples[:, idx[1]]
    e1_chain = samples[:, idx[2]]
    e2_chain = samples[:, idx[3]]
    e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
    q_chain = (1 - e_chain) / (1 + e_chain)
    phi_chain = 0.5 * np.arctan2(e2_chain, e1_chain) * 180 / np.pi
    return r_chain, n_chain, q_chain, phi_chain

def summarize_chain(chain):
    med, lo, up = get_median_and_uncertainties(chain)
    return format_val(med, lo, up)

# --- LaTeX header ---
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{Median lens light parameters computed from the best-fit model. "
      "The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}")
print("\\label{tab:lens_light_params}")
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lccccc}")
print("\\hline")
print("Lens System & Filter & $R_{\\text{S\\'ersic}}$ [arcsec] & "
      "$n_{\\text{S\\'ersic}}$ & $q$ & $\\phi_{\\text{ext}}$ [deg] \\\\")
print("\\hline")

# --- Loop over systems ---
for name in names:
    row_started = False
    for filt in ["F814W", "F475X", "F160W"]:
        filename = f"../cutout_data/{name}/{filt}/{name}_{filt}.pkl"
        if not os.path.exists(filename):
            if not row_started:
                print(f"{name} & {filt} & -- & -- & -- & -- \\\\")
                row_started = True
            else:
                print(f" & {filt} & -- & -- & -- & -- \\\\")
            continue

        with open(filename, "rb") as f:
            loaded_data = pickle.load(f)
        chain_list = loaded_data.get('chain_list')
        sampler_type, samples, param_mcmc, dist_mcmc = chain_list[1]

        # number of profiles
        if filt == "F160W":
            n_profiles = 2  # always two
        elif name == "J0602-4335":
            n_profiles = 2
        else:
            n_profiles = 1

        for prof_idx in range(n_profiles):
            param_names = [
                f'R_sersic_lens_light{prof_idx}',
                f'n_sersic_lens_light{prof_idx}',
                f'e1_lens_light{prof_idx}',
                f'e2_lens_light{prof_idx}'
            ]
            r_chain, n_chain, q_chain, phi_chain = extract_profile(samples, param_names, param_mcmc)

            r_str = summarize_chain(r_chain)
            n_str = summarize_chain(n_chain)
            q_str = summarize_chain(q_chain)
            phi_str = summarize_chain(phi_chain)

            # Print row
            if not row_started:
                print(f"{name} & {filt} & {r_str} & {n_str} & {q_str} & {phi_str} \\\\")
                row_started = True
            else:
                print(f" & {filt} & {r_str} & {n_str} & {q_str} & {phi_str} \\\\")

    print("\\hline")

print("\\end{tabular*}")
print("\\end{table*}")


\begin{table*}[htb]
\centering
\caption{Median lens light parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}
\label{tab:lens_light_params}
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lccccc}
\hline
Lens System & Filter & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $\phi_{\text{ext}}$ [deg] \\
\hline
J0407-5006 & F814W & $0.40^{+0.01}_{-0.01}$ & $4.85^{+0.15}_{-0.13}$ & $0.66^{+0.01}_{-0.01}$ & $-8.26^{+0.89}_{-0.88}$ \\
 & F475X & $9.92^{+0.06}_{-0.14}$ & $5.96^{+0.03}_{-0.05}$ & $0.79^{+0.04}_{-0.04}$ & $-2.82^{+8.44}_{-6.78}$ \\
 & F160W & $0.57^{+0.02}_{-0.02}$ & $5.84^{+0.11}_{-0.17}$ & $0.59^{+0.01}_{-0.01}$ & $-16.02^{+0.39}_{-0.38}$ \\
 & F160W & $3.65^{+0.44}_{-0.33}$ & $1.02^{+0.17}_{-0.14}$ & $0.58^{+0.02}_{-0.02}$ & $31.81^{+3.94}_{-3.83}$ \\
\hline
J0602-4335 & F814W & $0.62^{+0.00}_{-0.00}$ & $0.81^{+0.02}_{-0.02}$ & $0.11^{+0.00}_{-0.00}$ & $-45.98^{+

# Lens MASS

In [5]:
import pickle
import numpy as np
import os

names = ['J0407-5006', 'J0602-4335', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filters = ['F814W', 'F475X', 'F160W']

# LaTeX table header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{Median lens mass parameters computed from the best-fit model. "
      "The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}")
print("\\label{tab:lens_mass_params}")
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lccccc}")
print("\\hline")
print("Lens System & Filter & $\\theta_E$ [arcsec] & $q_{\\text{mass}}$ & $\\phi_{\\text{mass}}$ [deg] & $\\gamma$ \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for name in names:
    first_row = True
    for filt in filters:
        filename = f"../cutout_data/{name}/{filt}/{name}_{filt}.pkl"
        if not os.path.exists(filename):
            continue

        with open(filename, "rb") as f:
            loaded_data = pickle.load(f)
        chain_list = loaded_data.get('chain_list')
        sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list[1]

        param_names_mass = ['theta_E_lens0', 'e1_lens0', 'e2_lens0', 'gamma1_lens1', 'gamma2_lens1']
        indices = [param_mcmc.index(p) for p in param_names_mass]

        theta_E_chain = samples_mcmc[:, indices[0]]
        e1_chain = samples_mcmc[:, indices[1]]
        e2_chain = samples_mcmc[:, indices[2]]
        gamma1_chain = samples_mcmc[:, indices[3]]
        gamma2_chain = samples_mcmc[:, indices[4]]

        e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
        q_mass_chain = (1 - e_chain) / (1 + e_chain)

        gamma_chain = np.sqrt(gamma1_chain**2 + gamma2_chain**2)
        phi_ext_chain = 0.5 * np.arctan2(e2_chain, e1_chain) * 180 / np.pi

        theta_E_median, theta_E_lower, theta_E_upper = get_median_and_uncertainties(theta_E_chain)
        q_mass_median, q_mass_lower, q_mass_upper = get_median_and_uncertainties(q_mass_chain)
        gamma_median, gamma_lower, gamma_upper = get_median_and_uncertainties(gamma_chain)
        phi_ext_median, phi_ext_lower, phi_ext_upper = get_median_and_uncertainties(phi_ext_chain)

        theta_E = f"${theta_E_median:.2f}^{{+{theta_E_upper:.2f}}}_{{-{theta_E_lower:.2f}}}$"
        q_mass = f"${q_mass_median:.2f}^{{+{q_mass_upper:.2f}}}_{{-{q_mass_lower:.2f}}}$"
        gamma = f"${gamma_median:.2f}^{{+{gamma_upper:.2f}}}_{{-{gamma_lower:.2f}}}$"
        phi_ext = f"${phi_ext_median:.1f}^{{+{phi_ext_upper:.1f}}}_{{-{phi_ext_lower:.1f}}}$"

        if first_row:
            print(f"{name} & {filt} & {theta_E} & {q_mass} & {phi_ext} & {gamma} \\\\")
            first_row = False
        else:
            print(f" & {filt} & {theta_E} & {q_mass} & {phi_ext} & {gamma} \\\\")
    print("\\hline")

print("\\end{tabular*}")
print("\\end{table*}")


\begin{table*}[htb]
\centering
\caption{Median lens mass parameters computed from the best-fit model. The associated uncertainties are statistical and were calculated using the 84th and 16th percentiles.}
\label{tab:lens_mass_params}
\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lccccc}
\hline
Lens System & Filter & $\theta_E$ [arcsec] & $q_{\text{mass}}$ & $\phi_{\text{mass}}$ [deg] & $\gamma$ \\
\hline
J0407-5006 & F814W & $0.82^{+0.01}_{-0.01}$ & $0.80^{+0.07}_{-0.06}$ & $-68.1^{+15.3}_{-11.2}$ & $0.08^{+0.02}_{-0.03}$ \\
 & F475X & $1.00^{+0.01}_{-0.01}$ & $0.58^{+0.01}_{-0.01}$ & $32.4^{+1.4}_{-1.9}$ & $0.20^{+0.01}_{-0.01}$ \\
 & F160W & $0.72^{+0.01}_{-0.01}$ & $0.66^{+0.04}_{-0.04}$ & $-10.8^{+5.0}_{-4.0}$ & $0.10^{+0.01}_{-0.01}$ \\
\hline
J0602-4335 & F814W & $1.00^{+0.01}_{-0.01}$ & $0.92^{+0.03}_{-0.05}$ & $-54.6^{+110.0}_{-19.3}$ & $0.11^{+0.02}_{-0.02}$ \\
 & F475X & $0.89^{+0.01}_{-0.01}$ & $0.58^{+0.03}_{-0.04}$ & $-80.7^{+3.2}_{-3.4}$ & $0.17^{+0.02}_{-0.01}$ \\
 

# Fermat Potential and Time Delays

In [6]:
import h5py
import numpy as np

names_f814w = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']

z_l = [0.573, 0.284, 0.398, 0.4]
z_s = [1.54, 2.593, 1.158, 2.739]

# LaTeX table header for F814W only
print("\\begin{table}[htb]")
print("\\centering")
print("\\caption{blah blah}")
print("\\label{tab:cosmo_results}")
print("\\begin{tabular}{lcccc}")
print("\\hline")
print("Lens System & $z_l$ & $z_s$ & $\\Delta \\Phi_{AB}$ [arcsec$\\^2$] & $\\Delta t_{AB}$ [days] \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for i, name in enumerate(names_f814w):
    filename_f814w = f"../cutout_data/{name}/F814W/{name}_F814W_cosmo.hdf5"

    with h5py.File(filename_f814w, "r") as f:
            kwargs = {}
            for key in f:
                kwargs[key] = f[key][()]

    dphi_chain = kwargs['dphi_list']
    dt_chain = kwargs['dt_list']

    dphi_median, dphi_lower, dphi_upper = get_median_and_uncertainties(dphi_chain)
    dt_median, dt_lower, dt_upper = get_median_and_uncertainties(dt_chain)

    z_l_fmt = f"${z_l[i]}$"
    z_s_fmt = f"${z_s[i]}$"
    dphi_fmt = f"${dphi_median:.3f}^{{+{dphi_upper:.3f}}}_{{-{dphi_lower:.3f}}}$"
    dt_fmt = f"${dt_median:.1f}^{{+{dt_upper:.1f}}}_{{-{dt_lower:.1f}}}$"

    print(f"{name} & {z_l_fmt} & {z_s_fmt} & {dphi_fmt} & {dt_fmt} \\\\")

print("\\hline")
print("\\end{tabular}")
print("\\end{table}")


\begin{table}[htb]
\centering
\caption{blah blah}
\label{tab:cosmo_results}
\begin{tabular}{lcccc}
\hline
Lens System & $z_l$ & $z_s$ & $\Delta \Phi_{AB}$ [arcsec$\^2$] & $\Delta t_{AB}$ [days] \\
\hline
J0806+2006 & $0.573$ & $1.54$ & $-0.468^{+0.023}_{-0.010}$ & $-53.3^{+2.6}_{-1.2}$ \\
J1442+4055 & $0.284$ & $2.593$ & $-0.544^{+0.004}_{-0.006}$ & $-21.4^{+0.2}_{-0.2}$ \\
J1620+1203 & $0.398$ & $1.158$ & $2.453^{+0.071}_{-0.080}$ & $183.2^{+5.3}_{-6.0}$ \\
J2325-5229 & $0.4$ & $2.739$ & $-0.712^{+0.031}_{-0.024}$ & $-41.5^{+1.8}_{-1.4}$ \\
\hline
\end{tabular}
\end{table}
